# Home Assignment No. 2: Practice

To solve this task efficiently, here are some practical suggestions:

* You are **HIGHLY RECOMMENDED** to read relevant documentation, e.g. for [python](https://docs.python.org/3/), [numpy](https://docs.scipy.org/doc/numpy/reference/), [matlpotlib](https://matplotlib.org/) and [sklearn](https://scikit-learn.org/stable/). Also remember to use tutorials, lecture slides, and other resources.


* Instead of rewriting existing code try to use **BUILT-IN METHODS** available in the libraries.


* To complete this part of the homework, you have to write some **CODE** directly inside the specified places in the notebook **CELLS**.


* In some problems you are asked to provide a short discussion of the results. In these cases you have to create a **MARKDOWN** cell with your comments right after the corresponding code cell.


* For every separate problem, you can get **INTERMEDIATE scores**.


* Your **SOLUTION** notebook **MUST BE REPRODUCIBLE**, i.e. if a reviewer executes your code, the output will be the same (with all the corresponding plots) as in your uploaded notebook. For this purpose, we suggest to fix random `seed` or (better) define `random_state=` inside every algorithm that uses some pseudorandomness.


* Your code must be readable. For this purpose, try to include **necessary** (and not more) comments inside the code. But remember: **GOOD CODE MUST BE SELF-EXPLANATORY**.


* Many `sklearn` algorithms support multithreading (Ensemble Methods, Cross-Validation, etc.). Check if the particular algorithm has `n_jobs` parameter and set it to `-1` to use all the cores.


* In the end you need to hand in a **single zip file** containing **two notebooks** (theory and practice) as well as the **html exported version** of this notebook (**4 files in total**).


To begin let's import the essential (for this assignment) libraries.

In [ ]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import time

# Set default parameters for plots
plt.rcParams['figure.figsize'] = (12.0, 6.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

## Part 1: Linear SVM (face detection)

In this part you will need to implement a linear SVM classifier via the gradient descent algorithm.

First, let us have a look at the data we want to train our algorithm on.

In [ ]:
# Load the faces data
data = loadmat('faces.mat')
labels = np.squeeze(data['Labels'])
data = data['Data']

Let us visualize some examples from the dataset. The dataset contains a set of $24 \times 24$ grayscale images labeled as face/non-face. Our goal is to train a classifier on this data. For calculations we will treat the samples as flattened vectors, the same way as they are stored. For visualization one needs to reshape the samples to $24 \times 24$ images.

In [ ]:
# Visualize some examples from the dataset.
samples_per_class = 10
classes = [-1, 1]
train_imgs = np.reshape(data, [-1, 24, 24], order='F')

for y, cls in enumerate(classes):
    idxs = np.flatnonzero(np.equal(labels, cls))
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = y * samples_per_class + i + 1
        plt.subplot(len(classes), samples_per_class, plt_idx)
        plt.imshow(train_imgs[idx])
        plt.axis('off')
        plt.title(cls)
plt.show()

## Task 1.1: SVM loss function [6 points]

In the lectures we have seen the SMO algorithm, which works in the dual domain. In this assignment we explore **Pegasos**, an optimization algorithm in the primal domain.

Recall the formulation of the SVM optimization problem as follows:

\begin{aligned}
& \min_{w, b}
& & \frac{1}{2}||w||^2 + C\sum_{i=1}^m \xi_i \\
& \ \text{ s.t.}
& & y^{(i)}(w^Tx^{(i)}+b) \geq 1-\xi_i, \; i = 1, \ldots, m \\
& & & \xi_i \geq 0, \; i = 1, \ldots, m \\
\end{aligned}

Let $f(x)=w^Tx+b$. The constraints can then be written as $y^{(i)}f(x^{(i)})\geq 1-\xi_i$. Together with the constraints $\xi_i \geq 0$ this leads to $\xi_i=\max(0, 1-y^{(i)}f(x^{(i)}))$. The above constraint optimization problem is therefore equivalent to the following **unconstrained** problem:

\begin{equation}
\min_{w, b} \frac{\lambda}{2}||w||^2 + \frac{1}{m}\sum_{i=1}^m \max(0, 1-y^{(i)}f(x^{(i)}))
\end{equation}

The first term in this objective is a regularization term (prevents overfitting) and the second term measures the classification loss. Here the parameter $\lambda=1/C$ is a **hyper-parameter** that controls the relative weight of  both losses.


**TODO**: Implement the **unconstrained** objective function for SVM in the following *svm_loss* function according to the specifications.

***HINT***: Write a small test for the provided values of w, b and C. Make sure your function passes the test.

In [ ]:
def svm_loss(w, b, X, y, lambda_):
    """
    Computes the loss of a linear SVM w.r.t. the given data and parameters

    Args:
        w: Parameters of shape [num_features]
        b: Bias (a scalar)
        X: Data matrix of shape [num_data, num_features]
        y: Labels corresponding to X of size [num_data, 1]
        lambda_: Regularization hyper-parameter

    Returns:
        l: The value of the objective for a linear SVM

    """

    #######################################################################
    # TODO:                                                               #
    # Compute and return the value of the unconstrained SVM objective     #
    #                                                                     #
    #######################################################################
    
    l = None

    #######################################################################
    #                         END OF YOUR CODE                            #
    #######################################################################
    return l


In [ ]:
# Test your cost-function
w_0 = np.zeros(data.shape[1])
b_0 = 0.
l_0 = svm_loss(w_0, b_0, data, labels, 1.)

#######################################################################
# TODO:                                                               #
# What should be the expected output of svm_loss with the given params#
#                                                                     #
#######################################################################


expected_output = None

#######################################################################
#                         END OF YOUR CODE                            #
#######################################################################

assert np.allclose(l_0, expected_output), "Something is wrong"
print("Well done!")

## Task 1.2: SVM gradient [9 points]

**TODO**: Implement the gradient of the above unconstrained objective w.r.t. to the parameters $w$ and $b$. The gradient will be computed on a mini-batch (i.e., a random subset of the training set).

**Hint**: Don't worry about the fact that $\max(0, 1-y^{(i)}f(x^{(i)}))$ is not differentiable at $1-y^{(i)}f(x^{(i)})=0$. Just pick a one-sided gradient (this is called a subgradient for convex functions).

In [ ]:
def svm_gradient(w, b, x, y, lambda_):
    """
    Compute gradient for SVM w.r.t. to the parameters w and b on a mini-batch (x, y)

    Args:
        w: Parameters of shape [num_features]
        b: Bias (a scalar)
        x: A mini-batch of training example [k, num_features]
        y: Labels corresponding to x of size [k]
        lambda_: Regularization hyper-parameter

    Returns:
        grad_w: The gradient of the SVM objective w.r.t. w of shape [num_features]
        grad_v: The gradient of the SVM objective w.r.t. b of shape [1]

    """




    #######################################################################
    # TODO:                                                               #
    # Compute the gradient for a particular choice of w and b.            #
    # Compute the partial derivatives and set grad_w and grad_b to the    #
    # partial derivatives of the cost w.r.t. both parameters in theta     #
    #                                                                     #
    #######################################################################

    grad_w = None
    grad_b = None

    #######################################################################
    #                         END OF YOUR CODE                            #
    #######################################################################
    return grad_w, grad_b

In [ ]:
# Test your implementation
x_ = np.ones([2, 10])
y_ = np.array([1, -1])
w_0 = np.zeros(10)
b_0 = 0.
grad_w, grad_b = svm_gradient(w_0, b_0, x_, y_, 1.)

#######################################################################
# TODO:                                                               #
# What should be the expected output of svm_gradient                  #
# with the given params                                               #
#                                                                     #
#######################################################################

expected_grad_w = None
expected_grad_b = None

#######################################################################
#                         END OF YOUR CODE                            #
#######################################################################

assert np.allclose(grad_w, expected_grad_w), "Something is wrong"
assert np.allclose(grad_b, expected_grad_b), "Something is wrong"
print("Well done!")

## Task 1.3: SVM Solver [18 points]

You will implement the **Pegasos** algorithm - a variant of SGD - to solve for the parameters $w$ and $b$. 

The algorithm was introduced in the following [Paper](http://ttic.uchicago.edu/~nati/Publications/PegasosMPB.pdf) (see Figure 2). It is essentially Stochastic Gradient Descent on mini-batches + a specific choice for the learning rate giving convergence guarantees. The required steps are outlined in the following class implementation. For more details, please refer to the [Paper](http://ttic.uchicago.edu/~nati/Publications/PegasosMPB.pdf).

**TODO**: Implement the Pegasos algorithm according to specs. Tune the hyper-parameter **C** to get at least 90% accuracy on test.

In [ ]:
class LinearSVM:
    def __init__(self, lambda_):
        self.lambda_ = lambda_
        self.w = None
        self.b = None
        
    def fit(self, X, y, num_iter=30000, num_per_batch=100, verbose=False):
        """
        Pegasos SVM solver.

        Args:
            X: Data matrix of shape [num_train, num_features]
            y: Labels corresponding to X of size [num_train]
            num_iter: Number of iterations
            num_per_batch: Number of samples per mini-batch

        Returns:
            theta: The value of the parameters after training

        """

        self.w = np.zeros(X.shape[1])
        self.b = 0.
        for t in range(num_iter):
            start = time.time()
            
            #######################################################################
            # TODO:                                                               #
            # Perform one step of stochastic gradient descent:                    #
            #   - Select a single training batch at random                        #
            #   - Update theta based on alpha and using gradient_function         #
            #                                                                     #
            #######################################################################
            
            # 1st Step: Sample a random mini-batch of size num_per_batch

            # 2nd Step: Compute the learning-rate n_t=1/(lambda*t)

            # 3rd Step: Compute the gradients and update the parameters as

            #######################################################################
            #                         END OF YOUR CODE                            #
            #######################################################################
            
            if verbose and t % 5000 == 0:
                exec_time = time.time() - start
                loss = svm_loss(self.w, self.b, X, y, self.lambda_)
                print(f'Iter {t}/{num_iter}: cost = {loss}  ({exec_time}s)')

    def predict_score(self, X):
        """
        Predicts a score for each sample (how confident the prediction is)
        
        Args:
            X: Data matrix of shape [num_train, num_features]
            
        Returns:
            Predictions of shape [num_train] - real numbers
        """
        #######################################################################
        # TODO:                                                               #
        # Write a function that predicts the "score" (condifence) for each    #
        # sample:                                                             #
        #                                                                     #
        #######################################################################
        
        preds = None

        #######################################################################
        #                         END OF YOUR CODE                            #
        #######################################################################

        return preds                
                
    def predict(self, X):
        """
        Predicts class labels on X.
        
        Args:
            X: Data matrix of shape [num_train, num_features]
            
        Returns:
            Predictions of shape [num_train]
        """
        #######################################################################
        # TODO:                                                               #
        # Write a function that predicts the class label {-1, 1} given the    #
        # data matrix X and trained params w and b. Use predict_score method  # 
        #                                                                     #
        #######################################################################

        preds = None

        #######################################################################
        #                         END OF YOUR CODE                            #
        #######################################################################

        return preds
    
    def score(self, X, y):
        """
        Calculates accuracy of the model on X.
        
        Args:
            X: Data matrix of shape [num_train, num_features]
            
        Returns:
            Model's accuracy score.
        """

        #######################################################################
        # TODO:                                                               #
        # Write a functions that calculates the performance of the model on X #
        # in terms of accuracy                                                #
        #                                                                     #
        #######################################################################
        
        score = None

        #######################################################################
        #                         END OF YOUR CODE                            #
        #######################################################################
        
        return score

Now we are going to train the model. First let's split the data into train and test sets and normalize them.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.3)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:


# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Now let us create a classifier instance and **fit** in on the training data.

In [ ]:
# We will measure the execution time
start = time.time()

C = 0.01
lambda_ = 1/C
my_svm = LinearSVM(lambda_=lambda_)
my_svm.fit(X_train, y_train, num_iter=30000, num_per_batch=64, verbose=True)

exec_time = time.time()-start
print('Total exection time: {}s'.format(exec_time))

 We can have a look at what theta has learned to recognise as "face"

In [ ]:
plt.imshow(np.reshape(my_svm.w, [24, 24], order='F'))
plt.title('Learned w')
plt.show()

Make the predictions, calculate the accuracy. Tune the hyperparameter C to get at least **90%** accuracy on the test!

In [ ]:
print('Found C = {}, lambda = {}'.format(C, lambda_))
print('Accuracy train: {}'.format(my_svm.score(X_train, y_train)))
print('Accuracy test: {}'.format(my_svm.score(X_test, y_test)))

## Part 2: Multiclass classification

## Task 2.1: ONE-VS-ALL SVM [12 points]

SVM is designed to solve a binary classification problem. What if we want to solve _multiclass classification_, meaning that we want to classify sample in 1 to K labels (with K>2)?

We can still use binary SVM to solve this problem in a _one-vs-all_ manner, meaning that we can train _K_ binary classifiers distinguishing class $k$ vs the rest of the classes.

You will implement multiclass classification using your LinearSVM implementation as follows:
1. Initialize MulticlassLinearSVM with _K_ LinearSVMs
2. For each _k=1..K_ labels:  
    a. Convert multiclass labels into binary labels: _y==1_ if _target==k_, _y==-1_ if _target!=k_  
    b. Train a binary classifier with this labels
3. To predict a label for an unknown sample:  
    a. Get a prediction score for each binary classifier  
    b. Return the label for which the score is the highest

**TODO**: Implement the multiclass SVM.

In [ ]:
class MulticlassLinearSVM:
    def __init__(self, lambda_, K_classes):
        self.lambda_ = lambda_
        self.K_classes = K_classes
        #######################################################################
        # TODO:                                                               #
        # Initialize the class with K_classes instances of LinearSVM          #
        #######################################################################
        self.classifiers = None
        
        
    def fit(self, X, y, num_iter=30000, num_per_batch=100, verbose=False):
        """
        Fit K classifiers

        Args:
            X: Data matrix of shape [num_train, num_features]
            y: Labels corresponding to X of size [num_train]; values from 0..K_classes-1
            num_iter: Number of iterations
            num_per_batch: Number of samples per mini-batch


        """

        #######################################################################
        # TODO:                                                               #
        # Train K_classes binary classifiers                                  #
        #   - Convert integer labels 0..K-1 to binary labels 1 / -1 for a given #
        #     binary classifier                                               #
        #   - Train the current binary classifier                             #
        #                                                                     #
        #######################################################################
        
        for k in range(0, self.K_classes):
            start = time.time()
            
            current_classifier = self.classifiers[k]
            
            #######################################################################
            # TODO:                                                               #
            # Convert multiclass labels y to binary labels classifying            #
            # y_bin = 1  if y==k vs y!=k                                          #
            # y_bin = -1 if y!=k vs y!=k                                          #
            #######################################################################


            
            #######################################################################
            # TODO:                                                               #
            # Train the binary classifier with the converted labels               #
            # Don't forget to use this method's arguments                         #
            #######################################################################
            

            #######################################################################
            #                         END OF YOUR CODE                            #
            #######################################################################
            
            exec_time = time.time() - start
            print(f'Classifier {k+1}/{self.K_classes}: time ({exec_time}s)')
    
    def predict(self, X):
        """
        Predicts class labels on X.
        
        Args:
            X: Data matrix of shape [num_train, num_features]
            
        Returns:
            Predictions of shape [num_train] in {0..K_classes-1}
        """
        #######################################################################
        # TODO:                                                               #
        # Write a function that predicts the class label {0, .., K_classes-1} #
        # Predict the score for every binary classifier (use predict_score)   #
        # Return the label for which the classifier gives the greatest score  #
        #                                                                     #
        #######################################################################       
        

        #######################################################################
        #                         END OF YOUR CODE                            #
        #######################################################################

        return preds
    
    def score(self, X, y):
        """
        Calculates accuracy of the model on X.
        
        Args:
            X: Data matrix of shape [num_train, num_features]
            
        Returns:
            Model's accuracy score.
        """

        #######################################################################
        # TODO:                                                               #
        # Write a functions that calculates the performance of the model on X #
        # in terms of accuracy                                                #
        #                                                                     #
        #######################################################################
        

        #######################################################################
        #                         END OF YOUR CODE                            #
        #######################################################################
        
        return score

### Simple (non-exhaustive) test
The code below should execute without errors and give ~25% accuracy

In [ ]:
C = 10.0
lambda_ = 1/C

n_classes = 4
n_samples = 2000
data_random = np.random.randn(n_samples, 2)
labels_random = np.array([np.random.randint(n_classes) for i in range(n_samples)])

mc_svm = MulticlassLinearSVM(lambda_=lambda_, K_classes=n_classes)
mc_svm.fit(data_random, labels_random, num_iter=1000, num_per_batch=64, verbose=True)

pred = mc_svm.predict(data_random)
print('Accuracy: ', mc_svm.score(data_random, labels_random))

exec_time = time.time()-start
print('Total execution time: {}s'.format(exec_time))

## Classification on FashionMNIST dataset
We'll train a multiclass SVM on FashionMNIST dataset, containing $28\times28$ grayscale images of 10 categories of clothing.  
Let's load and visualize the dataset

In [ ]:
fmnist_data = np.load('fashion_mnist.npz')
X_train, X_test, y_train, y_test = fmnist_data['x_train'], fmnist_data['x_test'], fmnist_data['y_train'], fmnist_data['y_test']

# Visualize some examples from the dataset.
samples_per_class = 10
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
train_imgs = np.reshape(X_train, [-1, 28, 28], order='F')

plt.figure(figsize=(20,20))
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(np.equal(y_train, cls))
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = y * samples_per_class + i + 1
        plt.subplot(len(classes), samples_per_class, plt_idx)
        plt.imshow(train_imgs[idx])
        plt.axis('off')
        plt.title(cls)
plt.show()

Now let's normalize the data, train the model and visualize the weights for each classifier

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
C = 0.01
lambda_ = 1./C
my_svm = MulticlassLinearSVM(lambda_=lambda_, K_classes=10)

my_svm.fit(X_train_scaled, y_train, num_iter=10000, num_per_batch=64, verbose=True)

exec_time = time.time()-start
print('Total exection time: {}s'.format(exec_time))

In [ ]:
print('C = {}, lambda = {}'.format(C, lambda_))
print('Accuracy train: {}'.format(my_svm.score(X_train_scaled, y_train)))
print('Accuracy test: {}'.format(my_svm.score(X_test_scaled, y_test)))

In [ ]:
for classifier in my_svm.classifiers:
    plt.figure(figsize=(3,3))
    plt.imshow(np.reshape(classifier.w, [28, 28], order='F'))
    plt.title('Learned w')
    plt.show()

**TODO:** Tune the parameter **C** to get at least **80%** test accuracy

In [ ]:
C = 
lambda_ = 1./C
my_svm = MulticlassLinearSVM(lambda_=lambda_, K_classes=10)

my_svm.fit(X_train_scaled, y_train, num_iter=10000, num_per_batch=64, verbose=True)

exec_time = time.time()-start
print('Total exection time: {}s'.format(exec_time))

In [ ]:
print('Found C = {}, lambda = {}'.format(C, lambda_))
print('Accuracy train: {}'.format(my_svm.score(X_train_scaled, y_train)))
print('Accuracy test: {}'.format(my_svm.score(X_test_scaled, y_test)))

## Part 3: SVM with RBF kernel

## Task 3.1: Compare RBF and linear kernels [10 points]

**TODO:** Compare SVM with linear and RBF kernels on toy datasets. Implement *plot_classifiers_predictions* function. Make sure to plot the decision regions as in the previous assignment as well as the support vectors.

**HINT:** You may use ideas from this [webpage](https://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane.html#sphx-glr-auto-examples-svm-plot-separating-hyperplane-py).


In [ ]:
from mlxtend.plotting import plot_decision_regions
from sklearn.datasets import make_moons, make_circles, make_blobs

In [ ]:
def plot_classifiers_predictions(X, y, classifiers):
    """
    Plots the decision regions and support vectors of the classifiers
    fit on X and y.

    Args:
        X: Data matrix of shape [num_train, num_features]
        y: Labels of shape [num_train]
        classifiers: A list of classfifier objects

    """

    fig, axes = plt.subplots(ncols=len(classifiers), nrows=1, figsize=(16, 8))

    for classifier, axis in zip(classifiers, axes.flat):
        #######################################################################
        # TODO:                                                               #
        # Implement the plotting function                                     # 
        #                                                                     #
        #######################################################################

        # Fit the classifier to the data

        # Plot the decision regions

        # Plot the support vectors

        #######################################################################
        #                         END OF YOUR CODE                            #
        #######################################################################
        
        axis.set_title('{}, accuracy = {:.2f}'.format(
            classifier.__class__.__name__, classifier.score(X, y)))

In [ ]:
from sklearn.svm import SVC

linear_svc = SVC(kernel="linear")
rbf_svc = SVC(kernel="rbf")

classifiers = [linear_svc, rbf_svc]

In [ ]:
X_blobs, y_blobs = make_blobs(n_samples=300, centers=[[-1.5, -1.5], [1.5, 1.5]])
plot_classifiers_predictions(X_blobs, y_blobs, classifiers)

In [ ]:
X_moons, y_moons = make_moons(n_samples=300, noise=0.2, random_state=0)
plot_classifiers_predictions(X_moons, y_moons, classifiers)

In [ ]:
X_circles, y_circles = make_circles(n_samples=300, noise=0.1, random_state=0, factor=0.6)
plot_classifiers_predictions(X_circles, y_circles, classifiers)

## Task 3.2: Questions [6 points]

* Play with the hyperparameters. How do the hyper-parameters influence the classifier? What happens for extreme values of the hyper-parameters?

***Your Answer:***

* Linear SVM vs. Gaussian Kernel SVM: Give advantages and disadvantages of both approaches. 

***Your Answer:***

* Linear SVM vs. Gaussian Kernel SVM: In what setting would you pick one method over the other? Answer in terms of number of training examples $m_{train}$ and feature dimension $d$

***Your Answer:***


## Part 4: Text classification

In this part you will be working on a text classification task. The IMDB dataset contains movie reviews labeled with positive/negative scores. Your goal here is to train a model to classify these texts.

Download the data from the [competition page](https://www.kaggle.com/t/a075874d324e402892eb5b4a31755fbb) and put it to your working directory.

We will use the *pandas* module to read, store and work with the data. The *pandas* module is a powerful library that provides us with a lot of useful functions and data structures that are designed to work with tabular data.

In [ ]:
import pandas as pd
# Load the data
imdb_dataset_train = pd.read_csv("train_imdb_dataset.csv", index_col=0)
reviews_test = pd.read_csv("test_reviews.csv", index_col=0)

In *pandas* the main data structure is the *pd.DataFrame*. It represents the data in a form of a table. You can access the rows using the indices and the columns with the column names. 

In [ ]:
imdb_dataset_train.head()

Let us have a look at the first row of the training *DataFrame*.


In [ ]:
i = 0
print("Sentiment:", imdb_dataset_train["sentiment"][i])
print("Review:", imdb_dataset_train["review"][i])

## Task 4.1: Preprocessing [3 points]

One needs to transform the data to the format that can be used with the known classifiers. First, let us map sentiments to the categorical labels.

**TODO:** Transform the *sentiments* to *labels*, an array containing 0s and 1s. You can use *OrdinalEncoder* from *sklearn.preprocessing*.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

#######################################################################
# TODO:                                                               #
# Transform sentiments to labels.                                     # 
#                                                                     #
#######################################################################

y_train = None

#######################################################################
#                         END OF YOUR CODE                            #
#######################################################################

Then we need to represent each text as a bag of words.

Using *CountVectorizer* from *sklearn.feature_extraction.text* we can transform the *reviews* to a data matrix *X* of shape [num_reviews, vocabulary_size], where each row represents a single text and each column indicates the number of occurences of a specific word across the dataset.
Notice that the Vectorizer has a lot of useful arguments. These could potentially influence the performance of the models. Make sure to at least set *stop_words="english"*. This will make the Vectorizer ignore frequent words, that are useless for classification (e.g. articles, prepositions... ). Make sure you fit the Vectorizer to the union of the training and test reviews, otherwise you might miss some of the words.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#######################################################################
# TODO:                                                               #
# Transform reviews to the words-frequency matrices                   #
# X_train and X_test                                                  #
#                                                                     #
#######################################################################

X_train = None
X_test = None

#######################################################################
#                         END OF YOUR CODE                            #
#######################################################################

In [ ]:
# Let us check the shape of the data matrix
assert X_train.shape[1] == X_test.shape[1], "Train and test matrices should have the same number of tokens"
print("Number of tokens found:", X_train.shape[1])
print("Well done!")

## Task 4.2: Multinomial Naive Bayes [3 points]

In this task you're asked to train a **Multinomial Naive Bayes** model on the reviews dataset. You're also asked to report the K-fold cross-validation. During the cross-validation the dataset gets split into K distinct subsets. At every iteration one of the subsets is removed from the training set, the model is trained on the rest and evaluated on the removed one. At the end the average of the K estimators is reported.

**TODO:** Train *MultinomialNB* model from *sklearn.naive_bayes*. Report the 5-fold cross-validation score.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

#######################################################################
# TODO:                                                               #
# Transform reviews to the words-frequency matrix X.                  # 
#                                                                     #
#######################################################################

cv_score = None

#######################################################################
#                         END OF YOUR CODE                            #
#######################################################################

In [ ]:
print("Cross-validation accuracy score: {:.2f}".format(cv_score))

Let us now train the model on the whole training dataset, make the predictions and save them to the submission file.

**TODO:** Train your model on the whole training set, make predictions and prepare a DataFrame for saving.

In [ ]:
#######################################################################
# TODO:                                                               #
# Train the model on X_train, y_train and make predictions for X_test.#
#                                                                     #
#######################################################################

predictions = None

#######################################################################
#                         END OF YOUR CODE                            #
#######################################################################

In [ ]:
#######################################################################
# TODO:                                                               #
# Create a submission DataFrame that has a single column "sentiment"  #
# With "positive"/"negative" values according to the predictions.     #
#                                                                     #
#######################################################################

submission = None

#######################################################################
#                         END OF YOUR CODE                            #
#######################################################################

submission["id"] = np.arange(submission.shape[0])
submission.to_csv("simple_baseline.csv", index=None)

In [ ]:
submission.head()

## Task 4.3: Time for the Competition! [10 points + 15 bonus]

In this task you need to improve the accuracy of the model as much as you can. Here are several techniques that you can use:

1) **Tune your hyper-parameters.** Try *GridSerachCV* function from *sklearn.model_selection* to find the best set of hyperparameters.

2) **Feature engineering.** Play with the representation of the textual data. We only tried one, but there are more (e.g. TF-IDF Vectorizer is another powerful method to transform text to a vector, taking into account the rareness of the words across the texts). Also do not hesitate to play with the arguments of the *Vectorizers*. 

3) **Change your model.** You are not restricted to train *Naive Bayes* only. You can use whatever algorithm you're already familiar with. Moreover, you can use the algorithms that you get to know during these 3 weeks of solving this assignment. E.g. give *RandomForests* a try!

When your model is trained, make the predictions for the test reviews and upload the file with those predictions to the [competition page](https://www.kaggle.com/t/a075874d324e402892eb5b4a31755fbb). You can find the formatting instructions to the submission file in the *Evaluation* section on the website.

After your submission is uploaded it will automatically get scored and will appear in the leaderboard. Before the deadline all the submissions are scored on 30% of the test set (*Public leaderboard*), after the deadline the submissions are rescored on the remaining part of the test set (*Private leaderboard*). The Private leaderboard may differ from the Public one. Notice, that the **final** order is accroding to the **Private leadearboard**. This is done to prevent you from overfitting to the Public leaderboard.

Notice that Kaggle limits you with **5** submissions per day, so make sure to test your model (e.g. with cross-validation) before you submit your predictions.

Make sure that the TAs can easily identify your submissions, i.e. don't use nicknames that cannot be linked to your name.

### Scoring rules
You get **3 points** if you beat the **simple baseline**, that is the classifier trained in task 4.2.

You get another **7 points** if you beat the **hard baseline**. This is the score of the hidden classifier that is trained by TAs, you don't have access to its code.

You get **5 bonus points** if you end up among **top-15** in the final leaderboard.

You get another **5 bonus points** if you end up in **top-10** in the final leaderboard.

And finally another **5 bonus points** will be given to those who end up in **top-5**.